# Introduksjon til PySCF

**PySCF (Python-based Simulations of Chemistry Framework)** er en åpen kildekode-pakke for kvantekjemi som er utviklet for å utføre en ulike elektronstrukturteoriberegninger (langt ord!). Det brukes mye til både forskning og undervisning på grunn av sin fleksibilitet, effektivitet og brukervennlighet. Navnet er et ordspill – "SCF" står vanligvis for Self-Consistent Field, altså Hartree-Fock og Tetthetsfunksjonalteori (DFT). Disse metodene er blant PySCFs sterkeste sider.

Dokumentasjonen er god, og et godt sted å starte er den offisielle nettsiden: https://pyscf.org/index.html


### Noen nøkkelfunksjoner i PySCF:
1. **Kjernefunksjonaliteter**:
   - PySCF støtter **Hartree-Fock (HF)**- og **Tetthetsfunksjonalteori (DFT)**-beregninger, samt avanserte metoder etter Hartree-Fock som **Møller-Plesset pertubasjonsteori (MP2)**, **Coupled Cluster (CC)**, **Konfigurasjonsinteraksjon (CI)**, **Complete Active Space Self-Consistent Field (CASSCF)**-beregninger og flere.

2. **Basis-sett og integraler**:
   - Programvaren inkluderer en rekke **Gaussiske orbitaler (GTO)** basis-sett og effektive rutiner for å beregne integraler.

3. **Modularitet og fleksibilitet**:
   - PySCF er designet for å være modulær, noe som gjør det mulig for brukere å utvide funksjonaliteten eller integrere den med andre verktøy. Dette gjør den egnet for skreddersydde arbeidsflyter i forskning.

4. **Python-grensesnitt**:
   - Skrevet i Python med ytelseskritiske deler i C, tilbyr PySCF et intuitivt og brukervennlig grensesnitt som utnytter Python-økosystemet for skripting og rask utvikling.
  

# En første beregning med Hartree-Fock

I denne delen av notebooken skal vi se hvordan vi definerer et molekyl, velger et GTO-basissett, og kjører en Hartree-Fock-beregning på molekylet.

## Importering av relevante pakker

In [86]:
from pyscf import * 
import matplotlib.pyplot as plt

## Definere et molekyl

Et molekyl defineres ved å plassere kjerner på posisjoner i rommet gitt ved XYZ-koordinater. PySCF regner ut hvor mange elektroner molekylet har under antagelsen at det er elektrisk nøytralt. Her definerer vi et vannmolekyl (med en litt rar geometri), og - for moro skyld - koffein-molekylet (bør være nær likevekt).

Når geometrien er bestemt, må man så velge et AO basissett. Husk at AO står for "atomic orbital". Her velger vi et basissett som heter STO-3G. Dette er en såkalt "minimal basis", der hvert elektron starter med 1 orbital, sentrert på atomet elektronet "hører til". I dette eksempelet vil det derfor være 1 basisfunksjon for hvert H-atom, og 5 basisfunksjoner for O-atomet. Hver basisfunksjon er en Slater-type orbital som er tilnærmet med 3 gaussiske orbitaler.

Så må vi be PySCF å "bygge" molekylet for å få en veldefinert datastruktur som PySCF kan gjøre beregninger med.

For mer informasjon og flere måter å gjøre dette på, se https://pyscf.org/user/gto.html.



In [87]:
water = """
O          0.00000        0.00000        0.00000
H          0.27740        0.89290        0.25440
H          0.60680       -0.23830       -0.71690

"""

caffeine = """
O          0.47000        2.56880        0.00060
O         -3.12710       -0.44360       -0.00030
N         -0.96860       -1.31250        0.00000
N          2.21820        0.14120       -0.00030
N         -1.34770        1.07970       -0.00010
N          1.41190       -1.93720        0.00020
C          0.85790        0.25920       -0.00080
C          0.38970       -1.02640       -0.00040
C          0.03070        1.42200       -0.00060
C         -1.90610       -0.24950       -0.00040
C          2.50320       -1.19980        0.00030
C         -1.42760       -2.69600        0.00080
C          3.19260        1.20610        0.00030
C         -2.29690        2.18810        0.00070
H          3.51630       -1.57870        0.00080
H         -1.04510       -3.19730       -0.89370
H         -2.51860       -2.75960        0.00110
H         -1.04470       -3.19630        0.89570
H          4.19920        0.78010        0.00020
H          3.04680        1.80920       -0.89920
H          3.04660        1.80830        0.90040
H         -1.80870        3.16510       -0.00030
H         -2.93220        2.10270        0.88810
H         -2.93460        2.10210       -0.88490
"""


mol = gto.Mole()
mol.atom = water
mol.basis = 'STO-3G'
mol.unit = 'Angstrom' # Alternativt 'Bohr'
mol.build()



## Kjøre en Hartree-Fock-beregning

Når vi har bestemt molekylet vårt, kan vi gjøre, for eksempel, en HF-beregning. Dette gjøres i to steg:
1. Lage et `RHF`-objekt som lar oss gjøre en Restricted Hartree-Fock-beregning. Dette objektet initialiseres med `Mole`-objektet fra tidligere.
2. Gjøre beregningene.



In [88]:
mf = scf.RHF(mol)  # Set up a Restricted Hartree-Fock solver
energy = mf.kernel()   # Solve for the energy

print("Energy: ", energy)

converged SCF energy = -74.9644475827696
Energy:  -74.9644475827696


## Valg av basissett

Kvantekjemi inneholder et veritabelt overflødighetshorn av GTO-baserte basissett, tilpasset mange ulike formål. I eksempelet over ble basissettet `STO-3G` brukt. Her er en tabell med flere basissett å prøve ut:

| **Basissett**    | **Type**        | **Beskrivelse**                                                                                                      | **Typiske bruksområder**                   |
|------------------|-----------------|----------------------------------------------------------------------------------------------------------------------|--------------------------------------------|
| **STO-3G**       | Minimalt Basis   | Bruker 3 Gauss-funksjoner for å tilnærme én Slater-type orbital. Rask, men med minimal nøyaktighet.                   | Små molekyler, enkle beregninger           |
| **3-21G**        | Split-Valence    | Splitter valensorbitalene i to Gauss-funksjoner for større fleksibilitet.                                             | Små systemer, moderat nøyaktighet          |
| **6-31G**        | Split-Valence    | Mer nøyaktig enn 3-21G; bruker 6 Gauss-funksjoner for kjernorbitaler og en delt valensorbitaltilnærming.              | Organiske molekyler, geometrioptimalisering|
| **cc-pVDZ**      | Korrelasjons-Konsistent | Dobbelt-zeta basissett designet for korrelerte bølgefunksjonsmetoder som CCSD og MP2.                               | Korrelerte beregninger, små molekyler      |
| **cc-pVTZ**      | Korrelasjons-Konsistent | Trippel-zeta versjon for bedre nøyaktighet, spesielt for korrelerte metoder.                                         | Nøyaktige korrelerte metoder               |
| **cc-pVQZ**      | Korrelasjons-Konsistent | Kvadrupel-zeta versjon for svært høy nøyaktighet i korrelerte beregninger.                                           | Høynøyaktige beregninger, referanseverdier |
| **def2-SVP**     | Split-Valence    | Dobbelt-zeta med polarisasjonsfunksjoner, optimalisert for DFT og korrelerte metoder.                                | Middels store molekyler, geometrioptimalisering |
| **def2-TZVP**    | Trippel-Zeta     | Trippel-zeta basissett med polarisasjon. Høy nøyaktighet, egnet for større systemer.                                  | DFT og bølgefunksjonsbaserte metoder       |
| **def2-QZVP**    | Kvadrupel-Zeta   | Kvadrupel-zeta basissett for ekstremt høy nøyaktighet, spesielt i post-HF metoder.                                    | Høynøyaktige beregninger                   |
| **LANL2DZ**      | ECP              | Bruker effektive kjernpotensialer (ECP) for tunge atomer, med dobbelt-zeta basissett for valenselektroner.            | Tunge elementer, overgangsmetaller         |
| **AUG-cc-pVDZ**  | Augmentert Basis | cc-pVDZ med diffuse funksjoner for å fange opp langdistanse-interaksjoner, bedre behandling av svakt bundne systemer. | Svake interaksjoner, anioner, eksiterte tilstander |
| **AUG-cc-pVTZ**  | Augmentert Basis | cc-pVTZ med diffuse funksjoner for mer nøyaktig behandling av eksiterte tilstander og anioner.                        | Svakt bundne systemer, van der Waals interaksjoner |


# Regne ut egenskaper til molekylet

Etter at HF-beregningen er ferdig, kan vi be PySCF om å regne ut ulike egenskaper. Her viser vi hvordan man regner ut dipolmomentet og Mulliken-populasjoner.


In [89]:
dip = mf.dip_moment()
print("Dipole moment: ", dip)
pop = mf.mulliken_pop()
print("Mulliken population: ", pop)



Dipole moment(X, Y, Z, Debye):  1.26968,  0.93998, -0.66413
Dipole moment:  [ 1.26967702  0.9399789  -0.66413293]
 ** Mulliken pop  **
pop of  0 O 1s            1.99768
pop of  0 O 2s            1.83424
pop of  0 O 2px           1.65696
pop of  0 O 2py           1.32455
pop of  0 O 2pz           1.54111
pop of  1 H 1s            0.82273
pop of  2 H 1s            0.82273
 ** Mulliken atomic charges  **
charge of    0O =     -0.35454
charge of    1H =      0.17727
charge of    2H =      0.17727
Mulliken population:  (array([1.99767794, 1.83424153, 1.65695672, 1.32454809, 1.54111392,
       0.82273109, 0.82273072]), array([-0.35453819,  0.17726891,  0.17726928]))


# Gjøre en DFT-beregning

Nå viser vi hvordan man kan gjøre en Kohn-Sham DFT-beregning i PySCF. Dette er ganske likt som Hartree-Fock-beregningen. Vi regner på koffein-molekylet.

Her er en tabell med XC-funksjonaler du kan prøve:

| **Funksjonal**            | **Definisjonsstreng** | **Beskrivelse**                                                                                          |
|---------------------------|-----------------------|----------------------------------------------------------------------------------------------------------|
| **LDA (Local Density Approximation)**        | `'lda'`               | Lokal tetthets-tilnærming. XC-funksjonalen avhenger bare av elektron-tettheten i hvert punkt.               |
| **PBE (Perdew-Burke-Ernzerhof)**  | `'pbe'`               | Generalisert gradient-tilnærming (GGA). Inkluderer gradienten av tettheten for mer nøyaktige beregninger.   |
| **BLYP (Becke-Lee-Yang-Parr)**     | `'blyp'`              | Kombinasjon av Becke's utvekslingsfunksjonal og LYP's korrelasjonsfunksjonal. GGA-nivå funksjonal.           |
| **BP86**                   | `'bp86'`              | Kombinasjon av Becke’s 1988 utvekslingsfunksjonal og Perdew's 1986 korrelasjonsfunksjonal.                  |
| **B3LYP**                  | `'b3lyp'`             | Hybrid GGA funksjonal som kombinerer BLYP med en andel av Hartree-Fock utvekslingsenergi.                   |
| **HSE (Heyd-Scuseria-Ernzerhof)**  | `'hse'`               | Hybrid GGA funksjonal med skjermet utveksling for forbedret behandling av systemer med lange rekkevidde.     |
| **M06**                    | `'m06'`               | Hybrid meta-GGA funksjonal, som inkluderer tetthetsgradienter og kinetisk energitetthet.                    |
| **TPSS (Tao-Perdew-Staroverov-Scuseria)** | `'tpss'`              | Meta-GGA funksjonal som tar hensyn til både elektron-tetthet og kinetisk energitetthet.                     |
| **SCAN**                   | `'scan'`              | Meta-GGA funksjonal som bruker strengt korrelerte tettheter for høyere nøyaktighet i komplekse systemer.    |
| **CAM-B3LYP**              | `'cam-b3lyp'`         | Hybrid funksjonal med lang-rekkevidde korreksjon, basert på B3LYP, men med tilleggsparameter for screening. |
| **PBE0**                   | `'pbe0'`              | Hybridversjon av PBE, som inkluderer en andel av Hartree-Fock utvekslingsenergi.                            |


In [90]:

mf_dft = scf.RKS(mol)

mf_dft.xc = 'b3lyp' # Velg en DFT-funksjonal

energy = mf_dft.kernel()
print("Energy: ", energy)

converged SCF energy = -75.3154254527387
Energy:  -75.31542545273867


# Geometrioptimering

Her viser vi hvordan vi kan gjøre en geometrioptimering. Da prøver PySCF å finne den geometrien som gir lavest energy med modellen og basissettet vi har valgt.


In [91]:
from pyscf.geomopt.geometric_solver import optimize
mol_eq = optimize(mf, maxsteps=100)
print(mol_eq.tostring())


                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********      .))))))  
          ##      .%%%%%%/      (%%%%%%,                  ,******      /)))))  
        %%%%%%      .%%%%%%#      *%%%%%%,    ,/////.       ******      )))))) 
      #%      %%      .%%%%%%/      *%%


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.277400   0.892900   0.254400    0.000000  0.000000  0.000000
   H   0.606800  -0.238300  -0.716900    0.000000  0.000000  0.000000
converged SCF energy = -74.9644475827708
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0315695267     0.0233709098    -0.0165136871
1 H    -0.0131555900    -0.0207130629     0.0005049672
2 H    -0.0184139367    -0.0026578469     0.0160087199
----------------------------------------------
cycle 1: E = -74.9644475828  dE = -74.9644  norm(grad) = 0.0549573


Step    0 : Gradient = 3.173e-02/4.261e-02 (rms/max) Energy = -74.9644475828
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 1.60000e-01 5.35840e-01 5.35841e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.026883  -0.019902   0.014062   -0.026883 -0.019902  0.014062
   H   0.294257   0.891121   0.237297    0.016857 -0.001779 -0.017103
   H   0.616826  -0.216619  -0.713859    0.010026  0.021681  0.003041

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -74.9656950490049
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0074364799    -0.0055051685     0.0038899764
1 H     0.0043922391     0.0004375825    -0.0039326418
2 H     0.0030442408     0.0050675861     0.0000426654
----------------------------------------------
cycle 2: E = -74.965695049  dE = -0.00124747  norm(grad) = 0.0130629


Step    1 : Displace = 2.873e-02/3.628e-02 (rms/max) Trust = 1.000e-01 (=) Grad = 7.542e-03/1.004e-02 (rms/max) E (change) = -74.9656950490 (-1.247e-03) Quality = 0.668
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 2.32592e-01 4.91194e-01 5.35840e-01


...


Geometry optimization cycle 4
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.019023  -0.014083   0.009951   -0.000553 -0.000409  0.000289
   H   0.288137   0.895733   0.245811    0.001340 -0.003446 -0.003280
   H   0.615086  -0.227050  -0.718262   -0.000787  0.003856  0.002990

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.

converged SCF energy = -74.9659003926598
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0006986629     0.0005172045    -0.0003654722
1 H    -0.0002407523    -0.0006314240    -0.0001374003
2 H    -0.0004579106     0.0001142196     0.0005028725
----------------------------------------------
cycle 4: E = -74.9659003927  dE = -2.67316e-05  norm(grad) = 0.00135659


Step    3 : Displace = 4.058e-03/4.942e-03 (rms/max) Trust = 1.000e-01 (=) Grad = 7.832e-04/9.430e-04 (rms/max) E (change) = -74.9659003927 (-2.673e-05) Quality = 0.881
Hessian Eigenvalues: 5.00000e-02 5.00000e-02 5.00000e-02 ... 2.64990e-01 5.35840e-01 6.17737e-01



Geometry optimization cycle 5
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -0.019350  -0.014325   0.010122   -0.000327 -0.000242  0.000171
   H   0.288242   0.896055   0.245898    0.000105  0.000322  0.000087
   H   0.615308  -0.227130  -0.718520    0.000222 -0.000080 -0.000258
converged SCF energy = -74.965901187568
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000541054    -0.0000400513     0.0000283036
1 H     0.0000245465     0.0000286257    -0.0000067657
2 H     0.0000295589     0.0000114257    -0.0000215380
----------------------------------------------
cycle 5: E = -74.9659011876  dE = -7.94908e-07  norm(grad) = 9.09313e-05


Step    4 : Displace = 3.822e-04/4.409e-04 (rms/max) Trust = 1.414e-01 (+) Grad = 5.250e-05/7.302e-05 (rms/max) E (change) = -74.9659011876 (-7.949e-07) Quality = 0.939
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    
Time elapsed since start of run_optimizer: 0.209 seconds


O          -0.01935       -0.01433        0.01012
H           0.28824        0.89605        0.24590
H           0.61531       -0.22713       -0.71852


# Beregne vibrasjonelle frekvenser

Som eksempel på hva PySCF kan gjøre viser vi hvordan man beregner vibrasjonelle frekvenser.

Her bruker vi den optimerte geometrien fra forrige beregning. Merk at modellen vår ikke er veldig nøyaktig, så de vibrasjonelle frekvensene er heller ikke veldig nøyaktige.


In [92]:
mf.mol = mol_eq
mf.kernel()

from pyscf import eph
myeph = eph.EPH(mf_dft)

mat, omega = myeph.kernel()

au_to_cmm1 = 219474.63 
print('Vibrasjonelle frekvenser i invers cm:')
print(omega * au_to_cmm1)

converged SCF energy = -74.8400504246501
Vibrasjonelle frekvenser i invers cm:
[4531.12894846 4214.91310955 1839.35172935]
